In [53]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pandas_market_calendars as mcal
from importlib import reload
import sys
import os
import time
from IPython.display import clear_output, display
from tqdm import tqdm

In [16]:
import functions; functions = reload(functions)
from functions import *

In [3]:
apikey = '3CenRhJBzNqh2_C_5S38pOyt3ozLvQDm'

In [4]:
# todo
# 1. test function for daily aggregates - none/errors/edge cases/before available
# 2. design pipeline for many jsons to csv
# 3. .db maybe?
# 4. if not, make csv
# 5. update/remake .py files to download and merge missing

In [5]:
date = '2020-01-01'
url = f'https://api.polygon.io/v2/aggs/grouped/locale/us/market/stocks/{date}?apiKey={apikey}'

In [6]:
response = requests.get(url)

In [7]:
response.json()

{'queryCount': 0,
 'resultsCount': 0,
 'adjusted': True,
 'status': 'OK',
 'request_id': 'a426fcea4aa5004a97f7d0e49adcb888'}

In [50]:
unauth = daily_agg('2018-01-02')
holiday = daily_agg('2020-01-01')
good = daily_agg('2020-06-01')

Attempted to request data past historical entitlements. Please upgrade your plan at https://polygon.io/pricing
No data available for 2020-01-01


In [17]:
mon = daily_agg('2024-06-03')
tue = daily_agg('2024-06-04')
wed = daily_agg('2024-06-05')

In [18]:
alljson = []
for json in [mon, tue, wed]:
    alljson.extend(json)

In [85]:
alldates = get_trading_days()
all = datelist_to_df(alldates)

In [147]:
import sqlite3

In [162]:
conn = sqlite3.connect('main.db')

In [163]:
all.head()

,T,v,vw,o,c,h,l,t,n
0,WB,981967.0,41.5953,41.47,41.61,41.93,41.02,1560801600000,11076.0
1,DLNGpB,14056.0,16.1327,16.51,16.00,16.51,15.90,1560801600000,102.0
2,WDC,4109647.0,36.3187,36.21,36.27,36.66,35.61,1560801600000,33259.0
3,GNTX,998643.0,23.4434,23.54,23.36,23.67,23.32,1560801600000,9896.0
4,BZQ,33935.5,104.5164,104.88,105.64,106.44,103.28,1560801600000,627.0


In [180]:
c = conn.cursor()

In [ ]:
c.execute('''
CREATE TABLE stonks (
    ticker TEXT,
    volume REAL,
    vw REAL,
    open REAL,
    close REAL,
    high REAL,
    low REAL,
    time INTEGER,
    n INTEGER
)
''')

In [189]:
info = c.execute('PRAGMA table_info(stonks)').fetchall()

In [190]:
info

[(0, 'ticker', 'TEXT', 0, None, 0),
 (1, 'volume', 'REAL', 0, None, 0),
 (2, 'vw', 'REAL', 0, None, 0),
 (3, 'open', 'REAL', 0, None, 0),
 (4, 'close', 'REAL', 0, None, 0),
 (5, 'high', 'REAL', 0, None, 0),
 (6, 'low', 'REAL', 0, None, 0),
 (7, 'time', 'INTEGER', 0, None, 0),
 (8, 'n', 'INTEGER', 0, None, 0)]

In [192]:
[i[1] for i in info]   

['ticker', 'volume', 'vw', 'open', 'close', 'high', 'low', 'time', 'n']

In [197]:
all.to_sql('stonks', conn, if_exists='append', index=False)

12678885